# Train Random Forest with RAPIDS

In [ ]:
import os
import time

import numpy as np
import cudf
import cuml
from cuml import RandomForestClassifier as cuRF
from sklearn.model_selection import train_test_split
from cuml.metrics.accuracy import accuracy_score
import airline_data_local

In [ ]:
print('cuDF version:', cudf.__version__)
print('cuML version:', cuml.__version__)

In [ ]:
data_dir = '/datasets/data_airline'

In [ ]:
# from airline_data import prepare_airline_data
# prepare_airline_data(data_dir, 20000000)

In [ ]:
%%time
df = cudf.read_parquet(os.path.join(data_dir, 'airline_20m.parquet'))

## Train model

In [ ]:
# Encode categoricals as numeric
for col in df.select_dtypes(['object']).columns:
    df[col] = df[col].astype('category').cat.codes.astype(np.float32)
        
# cast all remaining columns to float32
for col in df.columns:
    if col in df.select_dtypes(['object']).columns: pass
    else:
        df[col] = df[col].astype(np.float32)
            
df['ArrDelayBinary'] = 1 * (df['ArrDelay'] > 0)
    
X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()
y = df['ArrDelayBinary'].astype(np.int32).to_array()
del df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=77, test_size=0.2)

In [ ]:
%%time
cu_rf = cuRF(n_estimators=100, max_depth=20, n_bins=8, max_features='auto')
cu_rf.fit(X_train, y_train)

In [ ]:
cuml_pred = cu_rf.predict(X_test)

In [ ]:
cuml_accuracy = accuracy_score(cuml_pred, y_test) *100
print('Accuracy: cumlRF {:.2f}%'.format(cuml_accuracy))